In [1]:
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso, LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import sklearn.metrics.cluster as smc

In [2]:
brandTotalSales_df_raw = pd.read_csv(os.path.join('data','BrandTotalSales.csv'))
brandTotalUnits_df_raw = pd.read_csv(os.path.join('data','BrandTotalUnits.csv'))
brandAverageRetailPrice_df_raw = pd.read_csv(os.path.join('data','BrandAverageRetailPrice.csv'))
brandDetails_df_raw = pd.read_csv(os.path.join('data','BrandDetails.csv'))

# brand total sales

In [3]:
brandTotalSales_df_raw.head(5)

,Months,Brand,Total Sales ($)
0,09/2018,10x Infused,"1,711.334232"
1,09/2018,1964 Supply Co.,"25,475.21594500000"
2,09/2018,3 Bros Grow,"120,153.644757"
3,09/2018,3 Leaf,"6,063.5297850000000"
4,09/2018,350 Fire,"631,510.0481550000"


In [4]:
brandTotalSales_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25279 entries, 0 to 25278
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Months           25279 non-null  object
 1   Brand            25279 non-null  object
 2   Total Sales ($)  25279 non-null  object
dtypes: object(3)
memory usage: 592.6+ KB


In [5]:
brandTotalSales_df_raw.describe()

,Months,Brand,Total Sales ($)
count,25279,25279,25279
unique,37,1627,25277
top,05/2021,Lift Ticket Laboratories,0
freq,848,37,3


# brand total units

In [6]:
brandTotalUnits_df_raw.head(5)

,Brands,Months,Total Units,vs. Prior Period
0,#BlackSeries,08/2020,"1,616.3390040000000",NaN
1,#BlackSeries,09/2020,NaN,-1.000000
2,#BlackSeries,01/2021,715.5328380000000,NaN
3,#BlackSeries,02/2021,766.669135,0.071466
4,#BlackSeries,03/2021,NaN,-1.000000


In [7]:
brandTotalUnits_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27686 entries, 0 to 27685
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brands            27686 non-null  object 
 1   Months            27686 non-null  object 
 2   Total Units       25712 non-null  object 
 3   vs. Prior Period  24935 non-null  float64
dtypes: float64(1), object(3)
memory usage: 865.3+ KB


In [8]:
brandTotalUnits_df_raw.describe()

,vs. Prior Period
count,24935.000000
mean,0.265306
std,3.291373
min,-1.000000
25%,-0.351822
50%,-0.055216
75%,0.240113
max,250.792020


# brand average retail price

In [9]:
brandAverageRetailPrice_df_raw.head(5)

,Brands,Months,ARP,vs. Prior Period
0,#BlackSeries,08/2020,15.684913,NaN
1,#BlackSeries,09/2020,NaN,-1.000000
2,#BlackSeries,01/2021,13.611428,NaN
3,#BlackSeries,02/2021,11.873182,-0.127705
4,#BlackSeries,03/2021,NaN,-1.000000


In [10]:
brandAverageRetailPrice_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27211 entries, 0 to 27210
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brands            27211 non-null  object 
 1   Months            27211 non-null  object 
 2   ARP               25279 non-null  float64
 3   vs. Prior Period  24499 non-null  float64
dtypes: float64(2), object(2)
memory usage: 850.5+ KB


In [11]:
brandAverageRetailPrice_df_raw.describe()

,ARP,vs. Prior Period
count,25279.000000,24499.000000
mean,22.679732,-0.065028
std,19.802724,0.388923
min,0.000000,-1.000000
25%,10.512827,-0.088073
50%,17.033051,-0.011649
75%,31.505612,0.045232
max,700.874984,12.645741


# brand details

In [12]:
brandDetails_df_raw.head(5)

,State,Channel,Category L1,Category L2,Category L3,Category L4,Category L5,Brand,Product Description,Total Sales ($),...,Total THC,Total CBD,Contains CBD,Pax Filter,Strain,Is Flavored,Mood Effect,Generic Vendor,Generic Items,$5 Price Increment
0,California,Licensed,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),"1,103.964857",...,0,0,THC Only,NaN,Vanilla Frosting,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$10.00 to $14.99
1,California,Licensed,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),674.645211,...,0,0,THC Only,NaN,Vanilla Frosting,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$15.00 to $19.99
2,California,Licensed,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),"2,473.699102",...,0,0,THC Only,NaN,Blueberry Slushy,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$15.00 to $19.99
3,California,Licensed,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),"14,589.916417",...,0,0,THC Only,NaN,Blueberry Slushy,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$10.00 to $14.99
4,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Afghan Kush - Wax,145.39627,...,0,0,THC Only,NaN,Afghan Kush,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$35.00 to $39.99


In [13]:
brandDetails_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144977 entries, 0 to 144976
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   State                144977 non-null  object 
 1   Channel              144977 non-null  object 
 2   Category L1          144977 non-null  object 
 3   Category L2          144977 non-null  object 
 4   Category L3          144245 non-null  object 
 5   Category L4          102618 non-null  object 
 6   Category L5          50135 non-null   object 
 7   Brand                144977 non-null  object 
 8   Product Description  144977 non-null  object 
 9   Total Sales ($)      144977 non-null  object 
 10  Total Units          144977 non-null  object 
 11  ARP                  144977 non-null  float64
 12  Flavor               7807 non-null    object 
 13  Items Per Pack       144977 non-null  int64  
 14  Item Weight          64454 non-null   object 
 15  Total THC        

In [14]:
brandDetails_df_raw.describe()

,ARP,Items Per Pack
count,144977.000000,144977.000000
mean,30.828439,1.938259
std,19.367580,17.294108
min,0.000000,0.000000
25%,16.407796,0.000000
50%,28.073823,0.000000
75%,41.781699,0.000000
max,874.800010,1000.000000


# data cleaning

## fix months

In [15]:
# make months a date time
brandTotalSales_df_raw['Months'] = pd.to_datetime(brandTotalSales_df_raw['Months'])
brandTotalUnits_df_raw['Months'] = pd.to_datetime(brandTotalUnits_df_raw['Months'])
brandAverageRetailPrice_df_raw['Months'] = pd.to_datetime(brandAverageRetailPrice_df_raw['Months'])

## fix total sales

In [16]:
# make total sales floats
brandTotalUnits_df_raw['Total Units'] = pd.to_numeric(brandTotalUnits_df_raw['Total Units'].str.replace(',','').str[:8])

In [17]:
brandTotalUnits_df_raw.head(5)

,Brands,Months,Total Units,vs. Prior Period
0,#BlackSeries,2020-08-01,1616.3390,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000
2,#BlackSeries,2021-01-01,715.5328,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466
4,#BlackSeries,2021-03-01,NaN,-1.000000


In [18]:
brandTotalUnits_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27686 entries, 0 to 27685
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Brands            27686 non-null  object        
 1   Months            27686 non-null  datetime64[ns]
 2   Total Units       25712 non-null  float64       
 3   vs. Prior Period  24935 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 865.3+ KB


In [19]:
# make Total Sales ($) floats
brandTotalSales_df_raw['Total Sales ($)'] = pd.to_numeric(brandTotalSales_df_raw['Total Sales ($)'].str.replace(',','').str[:8])

In [21]:
brandTotalSales_df_raw.head(5)

,Months,Brand,Total Sales ($)
0,2018-09-01,10x Infused,1711.334
1,2018-09-01,1964 Supply Co.,25475.210
2,2018-09-01,3 Bros Grow,120153.600
3,2018-09-01,3 Leaf,6063.529
4,2018-09-01,350 Fire,631510.000


In [20]:
brandTotalSales_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25279 entries, 0 to 25278
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Months           25279 non-null  datetime64[ns]
 1   Brand            25279 non-null  object        
 2   Total Sales ($)  25279 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 592.6+ KB


# brand details categorical features

In [39]:
print('brand details\n')
for i, column in enumerate(brandDetails_df_raw.columns):
    print(column, '-'*5 + '>', len(list(brandDetails_df_raw[column].unique())))

brand details

State -----> 1
Channel -----> 1
Category L1 -----> 5
Category L2 -----> 13
Category L3 -----> 54
Category L4 -----> 72
Category L5 -----> 39
Brand -----> 1123
Product Description -----> 32608
Total Sales ($) -----> 133144
Total Units -----> 96910
ARP -----> 131319
Flavor -----> 496
Items Per Pack -----> 33
Item Weight -----> 70
Total THC -----> 136
Total CBD -----> 149
Contains CBD -----> 2
Pax Filter -----> 3
Strain -----> 5825
Is Flavored -----> 3
Mood Effect -----> 2
Generic Vendor -----> 2
Generic Items -----> 2
$5 Price Increment -----> 22


In [37]:
print('brand details\n')
for i, column in enumerate(brandDetails_df_raw.columns):
    uniq = list(brandDetails_df_raw[column].unique())
    if len(uniq) > 100:
        uniq = "too big: " + str(len(uniq))
    print(column, '\n', uniq, '\n')

brand details

State 
 ['California'] 

Channel 
 ['Licensed'] 

Category L1 
 ['Inhaleables', 'Topicals', 'Ingestibles', 'All Accessories', 'Other Cannabis'] 

Category L2 
 ['Flower', 'Concentrates', 'Pre-Rolled', 'Topicals', 'Edibles', 'Devices', 'Sublinguals', 'Other Cannabis', 'Accessories', 'Non Infused Food', 'Apparel', 'Grow Supplies', 'Shake/Trim/Lite'] 

Category L3 
 ['Hybrid', 'Sativa Dominant', 'Dabbable Concentrates', 'Infused Pre-Rolled', 'Pre-Rolled', 'Vape', 'Other Topicals', 'Indica', 'Sativa', 'Sativa Leaning', 'Infused Foods', 'Indica Dominant', 'Indica Leaning', 'Candy', 'Vaporizers', 'Pipe', 'Water Pipe', 'Rolling Papers', 'Pills', 'Tinctures', 'Other', 'Beverages', 'Spray', 'Grinder', 'Storage Device', 'Creams', 'Massage Oil', 'Balms/Salves', 'Bundles/Collections', 'Lighter', 'Culinary', 'Chocolates', 'Lotions', 'Suppositories', 'Dissolvable', 'Plants', 'Gum', nan, 'Other Edibles', 'Rolling Machine', 'Pet Products', 'Cleaner', 'Pre-Loaded', 'Lubricants', 'Soap', 

# null values

In [52]:
for i, column in enumerate(brandDetails_df_raw.columns):
    notNullL = len(brandDetails_df_raw.loc[brandDetails_df_raw[column].notnull()])
    l = len(brandDetails_df_raw)
    percent = notNullL/l
    s = percent
    if percent == 1:
        s = "no null vals"
    print(column, '-'*20+'>', s)

State --------------------> no null vals
Channel --------------------> no null vals
Category L1 --------------------> no null vals
Category L2 --------------------> no null vals
Category L3 --------------------> 0.9949509232498948
Category L4 --------------------> 0.7078226201397463
Category L5 --------------------> 0.3458134738613711
Brand --------------------> no null vals
Product Description --------------------> no null vals
Total Sales ($) --------------------> no null vals
Total Units --------------------> no null vals
ARP --------------------> no null vals
Flavor --------------------> 0.053849921021955204
Items Per Pack --------------------> no null vals
Item Weight --------------------> 0.44458086455092877
Total THC --------------------> no null vals
Total CBD --------------------> no null vals
Contains CBD --------------------> no null vals
Pax Filter --------------------> 0.3055726080688661
Strain --------------------> 0.7976368665374508
Is Flavored --------------------> 0.07

# time series feature engineering

In [53]:
brands = brandTotalUnits_df_raw['Brands'].unique()

In [54]:
brands

array(['#BlackSeries', '101 Cannabis Co.', '10x Infused', ..., 'Zlixir',
       'Zoma', 'Zuma Topicals'], dtype=object)

In [59]:
l = len(brands)
tempDf = []
for i, brand in enumerate(brands):
    print(f'{i}/{l-1}',end='\r')
    

featureEngineeredDf = pd.concat(tempDf)

ValueError: No objects to concatenate